# Storing Simulations to HDF

You can ask TARDIS to store the state of each iteration of the simulation you are running. We show examples of how this is done:

Initialize the simulation with the `tardis_example.yml` configuration file.

In [ ]:
from tardis.io.atom_data.util import download_atom_data
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation

# We download the atomic data needed to run the simulation
download_atom_data('kurucz_cd23_chianti_H_He')

# Must have the tardis_example folder in the working directory.
config_fname = 'tardis_example.yml'
config = Configuration.from_yaml(config_fname)

simulation = Simulation.from_config(config)

Run the simulation.

In [ ]:
simulation.run()

You can now use `to_hdf` method, to save properties to a HDF file.

#### Parameters  

`file_path`: Path where the HDF file should be stored. (Required)  
`path`: Path inside the HDF store to store the elements. (Optional)  
`name`: Name of the group inside HDF store, under which properties will be saved.(Optional)
`overwrite`: If the HDF file already exists, do you overwrite the existing file (Optional, default `False`)

<div class="alert alert-info">

Note
    
Throughout this notebook, we set ``overwrite=True`` so that the notebook can be run repeatedly if needed.

</div>

In [ ]:
simulation.to_hdf('/tmp/full_example.hdf', overwrite=True)

# The commented out code below shows an example of to_hdf with more parameters
#simulation.to_hdf(file_path='/tmp/full_example.hdf', path='/', name='simulation')

Open the stored HDF file with pandas and print its structure.

In [ ]:
import pandas as pd

data = pd.HDFStore('/tmp/full_example.hdf', overwrite=True)
print(data)

Access `model.homologous_density.density_0` under simulation, which is a one-dimensional array

In [ ]:
print(data['/simulation/model/homologous_density/density_0'])

Scalars are stored in a `scalars` `pandas.Series` for every module. For example to access `model.t_inner` under simulation, one would need to do the following.

Note: Quantities are always stored as their SI values.

In [ ]:
print(data['/simulation/model/scalars']['t_inner'])

## Breakdown of the various to_hdf methods
Every module in TARDIS has its own `to_hdf` method responsible to store its own data to an HDF file.

### Plasma
The following call will store every plasma property to `/tmp/plasma_output.hdf` under `/parent/plasma`

In [ ]:
simulation.plasma.to_hdf('/tmp/plasma_output.hdf', path='parent', overwrite=True)

In [ ]:
import pandas
with pandas.HDFStore('/tmp/plasma_output.hdf') as data:
    print(data)

Plasma's `to_hdf` method can also accept a `collection` parameter which can specify which types of plasma properties will be stored. For example if we wanted to only store Input plasma properties, we would do the following:

In [ ]:
from tardis.plasma.properties.base import Input
simulation.plasma.to_hdf('/tmp/plasma_input_output.hdf', collection=[Input], overwrite=True)

In [ ]:
import pandas
with pandas.HDFStore('/tmp/plasma_input_output.hdf') as data:
    print(data)

### Model
The following call will store properties of the `Radial1DModel` to `/tmp/model_output.hdf` under `/model`.

In [ ]:
simulation.model.to_hdf('/tmp/model_output.hdf', overwrite=True)

In [ ]:
import pandas
with pandas.HDFStore('/tmp/runner_output.hdf') as data:
    print(data)

### MontecarloRunner
The following call will store properties of the `MontecarloRunner` to `/tmp/runner_output.hdf` under `/runner`.

In [ ]:
simulation.runner.to_hdf('/tmp/runner_output.hdf', overwrite=True)

In [ ]:
import pandas
with pandas.HDFStore('/tmp/runner_output.hdf') as data:
    print(data)